In [130]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv


In [131]:
data_path = '/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv'
cc = pd.read_csv(data_path)
cc

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru
...,...,...,...,...,...,...,...,...,...
1790,Zotter,Peru,647,2011,70%,Austria,3.75,,Peru
1791,Zotter,Congo,749,2011,65%,Austria,3.00,Forastero,Congo
1792,Zotter,Kerala State,749,2011,65%,Austria,3.50,Forastero,India
1793,Zotter,Kerala State,781,2011,62%,Austria,3.25,,India


In [132]:
import re
for column in cc.columns:
    cc.columns = cc.columns.str.replace(r'\s*\n\s*', ' ', regex=True)
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Company (Maker-if known)          1795 non-null   object 
 1   Specific Bean Origin or Bar Name  1795 non-null   object 
 2   REF                               1795 non-null   int64  
 3   Review Date                       1795 non-null   int64  
 4   Cocoa Percent                     1795 non-null   object 
 5   Company Location                  1795 non-null   object 
 6   Rating                            1795 non-null   float64
 7   Bean Type                         1794 non-null   object 
 8   Broad Bean Origin                 1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [133]:
cc.rename(columns={
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'BarName',
    'Review Date':'ReviewDate',
    'Cocoa Percent':'CocoaPercent',
    'Company Location':'CompanyLocation',
    'Bean Type':'BeanType',
    'Broad Bean Origin' : 'BroadBeanOrigin',
}, inplace=True)

In [134]:
cc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1795 non-null   object 
 1   BarName          1795 non-null   object 
 2   REF              1795 non-null   int64  
 3   ReviewDate       1795 non-null   int64  
 4   CocoaPercent     1795 non-null   object 
 5   CompanyLocation  1795 non-null   object 
 6   Rating           1795 non-null   float64
 7   BeanType         1794 non-null   object 
 8   BroadBeanOrigin  1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [135]:
cc.isnull().sum()

Company            0
BarName            0
REF                0
ReviewDate         0
CocoaPercent       0
CompanyLocation    0
Rating             0
BeanType           1
BroadBeanOrigin    1
dtype: int64

In [136]:
cc['CocoaPercent'] = cc['CocoaPercent'].str.replace('%', '')
cc['CocoaPercent'] = cc['CocoaPercent'].astype(float)
cc['CocoaPercent'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1795 entries, 0 to 1794
Series name: CocoaPercent
Non-Null Count  Dtype  
--------------  -----  
1795 non-null   float64
dtypes: float64(1)
memory usage: 14.2 KB


In [146]:
cc['BeanType'] = cc['BeanType'].str.replace(' ', '', regex=True)
cc['BeanType'].unique()

array(['\xa0', 'Criollo', 'Trinitario', 'Forastero(Arriba)', 'Forastero',
       'Forastero(Nacional)', 'Criollo,Trinitario', 'Criollo(Porcelana)',
       'Blend', 'Trinitario(85%Criollo)', 'Forastero(Catongo)',
       'Forastero(Parazinho)', 'Trinitario,Criollo', 'CCN51',
       'Criollo(Ocumare)', 'Nacional', 'Criollo(Ocumare61)',
       'Criollo(Ocumare77)', 'Criollo(Ocumare67)', 'Criollo(Wild)',
       'Beniano', 'Amazonmix', 'Trinitario,Forastero',
       'Forastero(Arriba)ASS', 'Criollo,+', 'Amazon', 'Amazon,ICS', 'EET',
       'Blend-Forastero,Criollo', 'Trinitario(Scavina)',
       'Criollo,Forastero', 'Matina', 'Forastero(Arriba,CCN)',
       'Nacional(Arriba)', 'Forastero(Arriba)ASSS',
       'Forastero,Trinitario', 'Forastero(Amelonado)', nan,
       'Trinitario,Nacional', 'Trinitario(Amelonado)', 'Trinitario,TCGA',
       'Criollo(Amarru)'], dtype=object)

In [138]:
cc_numerik = cc.select_dtypes(include=np.number)
cc_numerik.info()
cc_object = cc.select_dtypes(include='object')
cc_object.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   REF           1795 non-null   int64  
 1   ReviewDate    1795 non-null   int64  
 2   CocoaPercent  1795 non-null   float64
 3   Rating        1795 non-null   float64
dtypes: float64(2), int64(2)
memory usage: 56.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Company          1795 non-null   object
 1   BarName          1795 non-null   object
 2   CompanyLocation  1795 non-null   object
 3   BeanType         1794 non-null   object
 4   BroadBeanOrigin  1794 non-null   object
dtypes: object(5)
memory usage: 70.2+ KB
